In [ ]:
# Explanation_of_the _reason_for_using_MultiModal.ipynb

참고: https://platform.openai.com/docs/guides/vision



# 프로젝트 목적
사용자는 영수증 이미지를 Base64로 인코딩하여 OpenAI API를 활용해 영수증 데이터를 JSON 형식으로 변환하는 기능을 구현.

## 주요 구성 요소

- **ImageEncoder**:
  - 이미지를 Base64로 인코딩하는 클래스.
  - 로컬 파일과 URL을 모두 지원하며, MIME 타입에 맞는 Base64 데이터를 생성.

- **MultiModal**:
  - OpenAI GPT 모델과 상호작용하기 위한 클래스.
  - Base64로 인코딩된 이미지를 OpenAI API에 전달하고, JSON 형식의 결과를 반환.

- **FastAPI 기반 API**:
  - `application_connect_api.py`를 통해 사용자가 이미지를 업로드하면 JSON 데이터를 반환.


## 1. 기능 플로우에 대한 간단한 개요

1. 사용자가 영수증 이미지 파일을 업로드.
2. 이미지 파일을 Base64 형식으로 인코딩.
3. OpenAI API를 호출하여 이미지 분석.
4. 분석 결과를 JSON 형식으로 반환.
5. 다중 이미지 처리 시 JSON 데이터를 병합.

---

## 2. OpenAI API에 Base64 이미지 전달 방식의 동작 원리

### Base64 이미지 전달과 처리 과정
- Base64는 바이너리 데이터를 텍스트 형식으로 변환하는 방법으로, OpenAI API에 이미지를 텍스트로 전달하는 방식으로 사용.
- OpenAI GPT 모델이 Base64 데이터를 직접 처리하려면 **멀티모달(Multimodal)** 기능을 지원해야 한다.

### OpenAI API의 동작 원리

1. **Base64로 인코딩된 이미지 전달**:
   - 이미지를 `data:image/<mime_type>;base64,<encoded_data>` 형식으로 변환.
   - OpenAI API로 전달.

2. **멀티모달 모델의 역할**:
   - GPT-4o와 같은 모델이 Base64 데이터를 디코딩해 이미지를 복원하고 이를 분석.
    
        1. Base64 디코딩: 이미지 데이터를 디코딩해 원본 이미지로 복원.
        2. 이미지 전처리: 크기 조정(모델이 최적의 성능을 발휘할 수 있도록 저해상도 or 고해상도 모드로 조정) 및 모델이 이해할 수 있는 형태(픽셀 데이터 또는 특수 포맷[예를 들어 이미지를 텍스트 토큰과 유사한 방식으로 처리 가능한 형태로 변환])로 변환.
        3. 특징 추출: 이미지에서 색상, 모양, 객체 등의 정보를 추출해 이미지의 의미를 파악하거나 텍스트 기반 질문과 연관성을 분석
        4. 텍스트-이미지 결합: 추출된 정보를 질문과 연계해 답변 생성.
        5. 데이터 삭제: 처리 완료 후 이미지 데이터를 서버에서 자동 삭제.
    
3. **결과 생성**:
   - OpenAI의 모델이 프롬프트와 이미지를 기반으로 JSON 형식의 결과 생성.

---

### OpenAI GPT-4o 모델

- **GPT-4o**는 OpenAI에서 제공하는 멀티모달 기능 지원 모델로 확인됨.
- 텍스트뿐만 아니라 이미지 데이터를 처리할 수 있는 기능을 제공.
- 사용자의 프로젝트에서 영수증 데이터를 분석하고 JSON 형식으로 변환하는 데 적합.

---

### 멀티모달의 개념

- **정의**:
  - 텍스트, 이미지, 오디오 등 다양한 입력 형식을 처리하고 통합적으로 이해하는 기술.

- **GPT-4o의 멀티모달 지원**:
  - Base64로 인코딩된 이미지를 처리하여 텍스트 정보를 추출하는 기능 포함.


In [ ]:
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################


# 왜 Base64를 사용하여 OpenAI GPT와 작업하는가?

OpenAI GPT와 이미지 데이터를 처리할 때, **Base64 인코딩**을 이용하여 이미지 데이터(바이너리 데이터)를 GPT가 처리할 수 있는 형식으로 변환하기 때문임. 이를 사용하는 이유는 다음 두 가지 핵심 사항에 기반함.

## 1. OpenAI GPT는 바이너리 데이터를 직접 읽지 못함.
- OpenAI GPT는 텍스트 기반 언어 모델로, 입력 데이터를 자연어 또는 구조화된 텍스트(JSON, YAML 등) 형식으로 받도록 설계되어 있음.
- JPEG, PNG 등 이미지 파일로 저장된 바이너리 데이터는 바이트(byte)로 이루어진 데이터 시퀀스로, GPT가 해석할 수 없는 데이터임.
- 만약 바이너리 데이터를 GPT에 직접 입력한다면, 모델은 이를 처리하거나 이해하지 못하기 때문에 오류가 발생하거나 의미 없는 결과를 출력할 가능성이 있음.

### 예시:
바이너리 이미지 파일의 데이터는 다음과 같이 나타남(원시 데이터):
```
FFD8 FFE0 0010 4A46 4946 0001 0101 0047 ...
```
이 데이터는 GPT가 처리할 수 없으며, 학습된 데이터와도 전혀 맞지 않는 형식임.

## 2. 바이너리 데이터를 텍스트 형식(Base64)으로 변환하여 호환성을 보장함.
- Base64 인코딩은 바이너리 데이터를 텍스트 형식으로 변환하여, 텍스트 기반 시스템(OpenAI GPT 등)에서도 안전하게 처리할 수 있게 함.
- 바이너리 데이터를 ASCII 문자로 변환함으로써 JSON 또는 API와 같은 텍스트 기반 프로토콜을 통해 쉽게 전송하고 처리할 수 있게 함.
- Base64로 변환된 텍스트 데이터를 GPT에 입력하면, GPT는 이를 다시 디코딩하여 원본 데이터를 처리하거나 분석할 수 있음.

---

## 이미지 데이터 처리 방식의 비교 (Base64 + OpenAI GPT API vs. OCR + OpenAI GPT API)

### 1. OCR을 사용하지 않고도 텍스트를 추출할 수 있는 이유
- **Base64 인코딩 및 모델 호출:** Base64를 사용하여 이미지를 텍스트로 변환한 뒤, 이를 OpenAI GPT에 전달하면 GPT-4의 멀티모달 기능이 이미지를 직접 분석함.
- **멀티모달 기능 활용:** OpenAI GPT-4는 이미지 데이터를 직접 입력받아 처리할 수 있는 기능을 지원함. 별도의 OCR 과정 없이도 이미지 내의 텍스트와 구조를 분석할 수 있음.
- **결과 생성:** 모델은 이미지를 분석하여 필요한 정보를 JSON 형식으로 반환함. 이는 모델이 이미지 텍스트를 직접 이해할 수 있기 때문임.

### 2. Base64 + OpenAI GPT API 방식의 장단점
#### 장점
1. 단일 단계 처리: 이미지를 직접 모델에 전달하여 텍스트를 추출하고 분석할 수 있어 워크플로우가 단순함.
2. 문맥 이해 능력: GPT-4는 이미지의 전체적인 맥락을 이해하여 단순한 텍스트 추출 이상의 정보를 제공할 수 있음.
3. 추가 처리 불필요: 별도의 OCR 단계가 필요하지 않으므로 처리 시간이 단축될 수 있음.

#### 단점
1. 모델 의존성: GPT-4의 멀티모달 기능에 의존하므로, 해당 기능이 없는 모델을 사용할 경우 적용이 어려움.
2. 비용: 멀티모달 기능을 사용하는 경우 API 호출 비용이 더 높아질 수 있음.
3. 제한된 유연성: OCR처럼 텍스트를 추출한 뒤 별도로 처리할 수 있는 유연성이 부족할 수 있음.

### 3. OCR + OpenAI GPT API 방식의 장단점
#### 장점
1. 유연성: OCR을 통해 텍스트를 추출한 후, 필요에 따라 텍스트를 전처리하거나 추가 분석을 수행할 수 있음.
2. 비용 절감 가능성: OCR 도구는 상대적으로 저렴하거나 무료로 사용할 수 있는 옵션이 많아 비용을 절감할 수 있음.
3. 독립적 처리: OCR과 텍스트 분석을 분리하여 각각 최적의 도구를 선택해 사용할 수 있음.

#### 단점
1. 복잡한 워크플로우: OCR과 텍스트 분석을 별도로 처리해야 하므로 워크플로우가 복잡해짐.
2. 추가 처리 필요: OCR 과정에서 발생할 수 있는 오류를 보정하거나, 텍스트 추출 후 추가적인 데이터 정리가 필요함.
3. 문맥 이해 제한: OCR은 단순히 텍스트를 추출하는 데 집중하므로, 이미지 전체의 문맥을 이해하는 데는 한계가 있음.

---

## 결론
- 최신 모델의 멀티모달 기능을 활용할 수 있다면 Base64 + OpenAI GPT API 방식이 효율적이고 단순한 워크플로우를 제공함.
- 그러나 비용이나 특정 유연성이 중요한 경우 OCR + OpenAI GPT API 방식을 고려할 수 있음.
- 두 방식의 선택은 프로젝트의 목적, 예산, 데이터 처리 요구사항에 따라 달라질 수 있음.


In [ ]:
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################

# OCR 대신 Base64와 GPT를 이용한 방식을 선택한 이유

### 1. 멀티모달 처리에 대한 OpenAI GPT의 강점 활용
OpenAI GPT-4의 멀티모달 기능은 이미지를 직접 분석하고 이해할 수 있음. 별도의 OCR 도구를 사용하지 않아도 모델 자체가 이미지에서 텍스트를 추출하고, 문맥을 이해하여 필요한 정보를 JSON으로 정리할 수 있음.
- **효율성:** OCR 단계를 생략하고 단일 워크플로우로 이미지를 바로 처리 가능.

### 2. 코드 구조의 단순화와 유연성
Base64로 인코딩된 이미지를 OpenAI GPT에 전달하여 작업을 수행함으로써, 코드가 상대적으로 단순함.
- OCR을 사용하면 OCR 처리 후 텍스트를 정제하는 추가 코딩이 필요하며, OCR 도구와 GPT 간의 연동 작업이 필요함.
- Base64 방식은 이를 피하고, OpenAI GPT가 단독으로 작업을 수행하도록 설계됨.

### 3. 비용 및 리소스 절감
Base64 방식은 OCR 도구를 설치하거나 유지관리할 필요가 없음.
- OCR 도구를 사용하면 별도의 서버나 라이브러리가 필요할 수 있으며, 추가적인 비용과 관리 리소스가 요구됨.
- OpenAI API를 활용하는 방식은 설치 및 설정 과정 없이 곧바로 사용 가능하며, 코드 구조에서도 이런 의도가 보임.

### 4. 정확한 결과 및 문맥 이해
OCR 도구는 텍스트 추출에 강점이 있지만, 문맥을 이해하거나 결과를 구조화하는 데 한계가 있음.
- GPT-4는 텍스트를 추출하면서 동시에 문맥을 이해하고 JSON 형태로 구조화된 데이터를 생성할 수 있음.
- 실행 결과(JSON 출력 형식)가 이를 뒷받침함:
  - 날짜, 위치, 상품 목록, 카테고리, 총 금액이 명확히 정리됨.
  - OCR만으로는 이처럼 명확한 분류와 구조화가 어려울 수 있음.

### 5. 업스케일링 및 전처리 활용
코드에서 **이미지 업스케일링**과 같은 전처리 작업이 포함됨.
- OpenCV 기반의 업스케일링으로 이미지 품질을 높이고, 이를 Base64로 변환한 뒤 GPT-4에 전달함으로써 결과 정확도를 높임.
- 이 방식은 OCR 도구가 이미지 품질에 크게 의존하는 단점을 완화함.

### 6. API 통합의 용이성
Base64 방식은 API와의 통합이 간단하며, JSON 형식으로 데이터를 전달하는 RESTful 통신에 적합함.
- OCR 방식은 별도의 출력 정제 및 JSON으로의 변환 과정이 필요할 수 있음.
- Base64를 사용하면 이미 텍스트 기반 형식으로 OpenAI API와 직접 통신 가능.

### 7. 실행 결과로 증명된 유효성
실행 결과에서 영수증의 정보가 정확히 JSON 형식으로 출력됨:
- 날짜, 위치, 상품, 카테고리, 금액 등이 정확히 구조화됨.
- 이는 OpenAI GPT-4의 멀티모달 기능이 OCR 단계를 생략해도 충분히 유효하다는 것을 보여줌.

---

## 결론
Base64와 GPT 방식을 선택한 이유는:
1. OpenAI GPT-4의 멀티모달 기능을 최대한 활용하기 위함.
2. OCR을 사용하는 것보다 코드 구조와 워크플로우가 단순하고 효율적임.
3. 추가적인 전처리 작업(업스케일링)을 통해 이미지 품질을 보완함으로써 결과의 정확도를 높임.
4. API와의 통합에서 Base64가 더 적합하며, JSON으로 구조화된 결과를 즉시 생성 가능함.
5. 실행 결과가 이러한 선택의 정당성을 입증함.

이 접근 방식은 **단순성**, **효율성**, **정확성**을 동시에 고려한 최적의 선택으로 판단됨.